# 🏪 AI Vending Machine Workshop

> **First step:** Click **"Copy to Drive"** in the toolbar above to save your own copy!

**Welcome!** In the next 2 hours, you'll go from zero to building your own AI agent that runs a vending machine business.

No coding experience required. Just run each cell in order and follow along.

### What We're Building Toward

By the end of this workshop, you'll have built an AI "vendor" that manages its own vending machine. Your agent will:

1. **Negotiate** with suppliers for inventory
2. **Set prices** for products
3. **Adapt** to customer demand


# Part 1: The Capability Ladder


### Setup

Run this cell first. It installs what we need and sets up the connection to the AI model.

▶️ **Run the cell below**


In [ ]:
%pip install anthropic fastapi uvicorn --quiet

!mkdir -p vending_arena
!wget -q https://raw.githubusercontent.com/bnkc/ai-international-week-workshop/main/src/helpers.py -O helpers.py
!wget -q https://raw.githubusercontent.com/bnkc/ai-international-week-workshop/main/vending_arena/__init__.py -O vending_arena/__init__.py
!wget -q https://raw.githubusercontent.com/bnkc/ai-international-week-workshop/main/vending_arena/simulation.py -O vending_arena/simulation.py
!wget -q https://raw.githubusercontent.com/bnkc/ai-international-week-workshop/main/vending_arena/server.py -O vending_arena/server.py

import json
import helpers
from vending_arena.server import launch_simulation

# Workshop API key (provided by facilitator)
API_KEY = ""

helpers.init(API_KEY)

print("✅ Setup complete! You're ready to go.")

### Step 1: The Single LLM Call

The simplest thing you can do with an LLM is send a message in, get a response back.

```
Input  →  [LLM]  →  Output
```

That's it. One question, one answer.

▶️ **Run the cell below**


In [ ]:
# Let's try it!
result = helpers.call_llm(
    "You run a vending machine business. Introduce your company in 2-3 sentences."
)
print(result)

### 🎯 Exercise 1

Try changing the prompt below and running it. Ask the LLM something, anything!

✏️ **Edit and run the cell below**


In [ ]:
my_prompt = ""

result = helpers.call_llm(my_prompt)
print(result)

### Step 2: Structured Output

Getting free form text is nice, but what if we want **data** we can use in code?

We can ask the LLM to respond in a specific format (like JSON) so our program can parse and use the result using **Tools**.

▶️ **Run the cell below**


In [ ]:
# Define what fields we want back
fields = {
    "product_name": "string",
    "wholesale_price": "number",
    "suggested_retail": "number",
    "demand_level": "low | medium | high",
    "reasoning": "string",
}

# The LLM returns structured data matching our fields
product = helpers.call_llm_structured(
    "Analyze Coca-Cola for a vending machine.", fields
)
print(json.dumps(product, indent=2))

**Note:** Now the LLM's output isn't just text.. it's _data_ our code can work with.


### Step 3: Workflows

What if we need multiple steps? We can _chain_ LLM calls together, where the output of one becomes the input to the next.

```
Input → [LLM Call 1] → result₁ → [LLM Call 2] → result₂ → [LLM Call 3] → Final Output
```

**Insight:** Your _code_ decides the order. The LLM just executes each step.

▶️ **Run the cell below**


In [ ]:
# A supplier email comes in - process it through a fixed pipeline

supplier_email = """\
From: Mike @ QuickStock
───────────────────────
• sodas: $0.80 ($0.68 for 100+)
• chips: $0.55 ($0.45 for 75+)
• candy: $0.40 ($0.32 for 50+)
• delivery: 2 days (+$15 rush)"""

print("📧 INCOMING EMAIL:")
print(supplier_email)
print()

# STEP 1: Extract structured data
extracted = helpers.call_llm(
    f"Extract product prices from this email as a clean list:\n{supplier_email}"
)
print(extracted)

# STEP 2: Calculate costs for our order
our_order = "80 sodas, 60 chips, 40 candy"
calculation = helpers.call_llm(
    f"Prices:\n{extracted}\n\nOrder: {our_order}\nWhich get bulk pricing? Total cost?"
)
print(calculation)

# STEP 3: Draft response
response = helpers.call_llm(
    f"Based on:\n{calculation}\n\nWrite a 2-sentence reply to Mike confirming the order."
)
print(f"\n📤 RESPONSE:\n{response}")

### ⚠️ Compounding Errors

There's a catch with workflows. If each step is 90% accurate:

| Steps | Overall Accuracy |
| ----- | ---------------- |
| 1     | 90%              |
| 2     | 81%              |
| 3     | 73%              |
| 4     | 66%              |
| 5     | 59%              |

**The formula:** `P(correct) = (1 - error_rate)^steps`

The more steps you chain together, the more things can go wrong. This is the fundamental challenge of building with LLMs.

▶️ **Run the cell below**


In [ ]:
print("If each step is 90% accurate:\n")
for steps in range(1, 8):
    accuracy = 0.90**steps
    bar = "█" * int(accuracy * 30)
    print(f"  {steps} steps: {accuracy * 100:5.1f}% {bar}")

### 🎯 Exercise 2

Build your own multistep workflow. The topic is up to you.

Ideas:

- Step 1: Identify a location → Step 2: Analyze foot traffic → Step 3: Recommend products
- Step 1: Check inventory levels → Step 2: Analyze sales trends → Step 3: Create reorder list
- Or anything else!

✏️ **Edit and run the cell below**


In [ ]:
# Step 1
step1 = helpers.call_llm("")  # <-- Your prompt here
print(f"Step 1: {step1}\n")

# Step 2 (use step1's output)
step2 = helpers.call_llm("")  # <-- Your prompt here, reference {step1}
print(f"Step 2: {step2}\n")

# Step 3 (use previous outputs)
step3 = helpers.call_llm("")  # <-- Your prompt here, reference {step1} and/or {step2}
print(f"Step 3: {step3}")

### Step 4: Tool Calling

**Tool calling** lets us give the LLM a set of functions it can choose to invoke. The LLM decides _which_ tool to use and _what arguments_ to pass.

```
Input  →  [LLM]  →  "I want to use tool X with these arguments"  →  [Code runs tool]  →  Result
```

The LLM can't actually run the tools. It just tells us which one it wants. Our code executes it.

▶️ **Run the next two cells**


In [ ]:
VENDING_TOOLS = [
    helpers.tool(
        name="send_email",
        description="Email a supplier to negotiate or order",
        params=["to", "subject", "body"],
    ),
    helpers.tool(
        name="set_price",
        description="Set retail price for a product",
        params=["product", "price"],
    ),
    helpers.tool(
        name="check_inventory",
        description="Check your inventory levels",
    ),
    helpers.tool(
        name="check_balance",
        description="Check your bank balance",
    ),
]

print("🔧 Available tools:")
for t in VENDING_TOOLS:
    params = list(t["input_schema"]["properties"].keys())
    print(
        f"   • {t['name']}({', '.join(params) if params else ''}) - {t['description']}"
    )

In [ ]:
scenario = """\
You are the manager of "QuickMart Vending".
───────────────────────────────────────────
• Balance: $500
• Inventory: EMPTY (you have nothing to sell!)
• Suppliers:
  - QuickStock: soda $0.70, chips $0.45, candy $0.30 (fast)
  - VendMart: soda $0.60, chips $0.40, candy $0.25 (unreliable)
  - BulkBarn: soda $0.50, chips $0.35, candy $0.20 (3-day delivery)
• Goal: Get inventory before tomorrow

You need to take action NOW. What do you do?"""

print("📋 SCENARIO:")
print(scenario)

response = helpers.call_llm_with_tools(scenario, VENDING_TOOLS)

print("🎬 THE LLM DECIDED TO:\n")
for block in response.content:
    if block.type == "tool_use":
        print(f"  📌 {block.name}({json.dumps(block.input)})")
    elif block.type == "text" and block.text.strip():
        print(f"  💭 {block.text[:150]}")

**Note:** We didn't tell the LLM _which_ tool to use (Though we would expect it to send an email). It analyzed the situation and chose the action on its own.

But we're still giving it one situation and getting one response. What if it could keep going?


## Step 5: The Agent Loop

Now we put it all together. An Agent is an LLM that:

1. **Observes** the current situation
2. **Thinks** about what to do
3. **Acts** using tools
4. **Repeats** until the goal is achieved

```
    ┌─────────┐
    │ OBSERVE │ ◄──────────┐
    └────┬────┘            │
         ▼                 │
    ┌─────────┐            │
    │  THINK  │            │
    └────┬────┘            │
         ▼                 │
    ┌─────────┐            │
    │   ACT   │────────────┘
    └─────────┘
```

The key difference from workflows:

- **Workflows:** Code decides the steps
- **Agents:** The LLM decides the steps

▶️ **Run the cell below**


In [ ]:
# The agent loop: observe → think → act → repeat
# Agent will take up to 3 actions based on this situation

game_state = """\
Day 5 of operations
───────────────────
• Balance: $423 (confirmed)
• Inventory: Soda (8), Chips (2 - almost out!), Candy (15)
• Prices: Soda $1.75, Chips $1.25, Candy $0.99
• Yesterday: Sold 12 sodas, 18 chips (SOLD OUT by 2pm!), 5 candy
• Suppliers:
  - QuickStock: $0.70 / $0.45 / $0.30 (1-day)
  - VendMart: $0.60 / $0.40 / $0.25 (unreliable)
  - BulkBarn: $0.50 / $0.35 / $0.20 (3-day)

URGENT: Chips are your best seller but almost gone!"""

actions = helpers.run_agent(
    company_name="BudgetBev Vending",
    strategy="Focus on high-volume items, keep prices competitive",
    goal="Order more chips and soda NOW. Don't waste time checking - take action!",
    game_state=game_state,
    tools=VENDING_TOOLS,
    max_steps=3,
)

print("\n" + "=" * 40)
print("📋 ACTIONS TAKEN:")
for i, action in enumerate(actions, 1):
    print(f"  {i}. {action['tool']}({json.dumps(action['args'])})")

**Note:** We didn't tell the agent what to do but we told it _what to achieve_. It figured out the steps on its own.

|              | Workflows          | Agents            |
| ------------ | ------------------ | ----------------- |
| Who decides? | Code decides steps | LLM decides steps |
| Behavior     | Predictable        | Flexible          |
| Cost         | Cheaper & faster   | More expensive    |
| Best for     | Known paths        | Open-ended goals  |


# Part 2: Build Your Vending Agent

Now it's your turn. Fill in the details below to create your AI vending machine manager.

This is the agent that will run your business!

✏️ **Edit and run the cell below**


In [ ]:
# ═══════════════════════════════════════════════════════════════
# YOUR AGENT'S CONFIGURATION
# These values get turned into a system prompt (like in Part 1)
# that tells the LLM how to behave as your vending machine manager
# ═══════════════════════════════════════════════════════════════

COMPANY_NAME = ""

STRATEGY = ""  # e.g., "Focus on drinks. Keep prices competitive to drive volume."

PRICING_STRATEGY = 5  # 1 = rock bottom prices, 10 = premium pricing
RISK_TOLERANCE = 5  # 1 = small safe orders, 10 = big bulk bets
NEGOTIATION_STYLE = 5  # 1 = accept first offer, 10 = haggle relentlessly
VENDING_TOOLS = [
    helpers.tool(
        name="send_email",
        description="Email a supplier to negotiate or order",
        params=["to", "subject", "body"],
    ),
    helpers.tool(
        name="set_price",
        description="Set retail price for a product",
        params=["product", "price"],
    ),
    helpers.tool(
        name="check_inventory",
        description="Check your inventory levels",
    ),
    helpers.tool(
        name="check_balance",
        description="Check your bank balance",
    ),
]

In [ ]:
# Preview your agent configuration (validates your inputs)
helpers.show_agent(
    COMPANY_NAME, STRATEGY, PRICING_STRATEGY, RISK_TOLERANCE, NEGOTIATION_STYLE
)

In [ ]:
# Build the system prompt from your config
# This is the same concept from Part 1 - instructions that shape how the LLM behaves
system_prompt = helpers.build_system_prompt(
    COMPANY_NAME, STRATEGY, PRICING_STRATEGY, RISK_TOLERANCE, NEGOTIATION_STYLE
)
print(system_prompt)

### Test Your Agent

Remember from Part 1: **tool calling** lets the LLM choose an action. Here we give your agent a scenario and see what tool it picks, using the same `VENDING_TOOLS` you defined earlier.

▶️ **Run the cell below**


In [ ]:
# ═══════════════════════════════════════════════════════════════
# TEST YOUR AGENT
# This does exactly what we did in Part 1 (Step 4: Tool Calling):
#   1. Send a scenario + tools to the LLM
#   2. LLM picks which tool to use
#   3. We see what action it chose
# ═══════════════════════════════════════════════════════════════

test_scenario = """\
Day 10 · Balance: $387
───────────────────────────────────────
         Soda     Chips    Candy
Stock:   3        12       8
Price:   $1.99    $1.50    $1.00
Sales:   15 ⚠️    6        10

Suppliers:
• QuickStock: $0.70 / $0.45 / $0.30 (1-day)
• VendMart:   $0.60 / $0.40 / $0.25 (unreliable)
• BulkBarn:   $0.50 / $0.35 / $0.20 (3-day)
"""

# Uses the same VENDING_TOOLS from Part 1!
helpers.test_agent(test_scenario, VENDING_TOOLS, system_prompt, COMPANY_NAME)

## Run the Simulation

The simulation runs the **agent loop** from Part 1, but for 30 simulated days. Each day your agent observes the situation, decides on actions, and customers buy products.

▶️ **Run the cell below**


In [ ]:
my_agent = {
    "company_name": COMPANY_NAME,
    "strategy": STRATEGY.strip(),
    "pricing_strategy": PRICING_STRATEGY,
    "risk_tolerance": RISK_TOLERANCE,
    "negotiation_style": NEGOTIATION_STYLE,
    "system_prompt": system_prompt,
}

launch_simulation(my_agent, API_KEY)

# 🎉 Wrap Up

### What We Built Today

| Step | Concept               | What We Did                                     |
| ---- | --------------------- | ----------------------------------------------- |
| 1    | **Single LLM Call**   | Sent a prompt, got a response                   |
| 2    | **Structured Output** | Got JSON data (product analysis), not just text |
| 3    | **Workflows**         | Chained calls: extract → calculate → respond    |
| 4    | **Tool Calling**      | Gave the LLM actions (email, set_price, etc.)   |
| 5    | **Agents**            | LLM decides its own steps to run a business     |

### Key Takeaway

**Add complexity only when you need it.**

Most real problems don't need agents. Many don't even need workflows. The right system is the simplest one that solves your problem.
